In [ ]:
data_path = '/Users/katyaartemova/data'
output_path = '/Users/katyaartemova/data'

lang = 'bar'

In [ ]:
def parse_cat(cat):
    return cat.split(':')[1]

In [ ]:
import pandas as pd
import json 

from tqdm.notebook import tqdm 
tqdm.pandas()

import feather

import wikipediaapi
import pywikibot
# import caffeine

In [ ]:
def split_sentences(text):
    sents = []
    for l in text.split('\n'):
        sents += [s+'.' if not s.endswith('.') else s for s in  l.split('.,')]
    if sents == ['.']:
        return None
    return sents

In [ ]:
import glob 
files = [f for f in glob.glob(f'{data_path}/{lang}_clean/*')]

dfs = []
for f in files:
    df =  pd.read_json(f, lines=True)
    dfs.append(df)
df = pd.concat(dfs)
    
df['sentences'] = df['text'].progress_apply(split_sentences)
df.dropna(subset = ['sentences'],inplace=True)                   

In [ ]:
langs = ['de']
def get_langlinks(page, langs = langs):
    records = {}
    langlinks = page.langlinks
 
    for k in langs:
        if k in langlinks.keys():
            v = langlinks[k]
            records[k] = v.title
        else:
            records[k] = None
    return records

wiki = wikipediaapi.Wikipedia(lang)

records = []
for idx, row in tqdm(df.iterrows(), total = len(df)):

    page = wiki.page(row['title'])
    record = get_langlinks(page)
    record[f'{lang}'] = row['title']
    records.append(record)
    if idx % 100 == 0:
        print(idx)

titles_df = pd.DataFrame.from_records(records)
titles_df.reset_index(drop = True, inplace = True)
titles_df.to_feather(f'{data_path}/{lang}_de_title')

In [ ]:
titles_df.to_feather(f'{data_path}/{lang}_de_title')